In [2]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [3]:
feature_number = 105

In [4]:

X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)


(393345, 105)
(393345,)


In [5]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.fit_transform(X)

In [6]:
pca1 = PCA(n_components= 0.999).fit(X_scaled)
pca2 = PCA(n_components= 0.95).fit(X_scaled)
X_pca1 = pca1.transform(X_scaled)
X_pca2 = pca2.transform(X_scaled)
print(X_pca1.shape)
print(X_pca2.shape)

(393345, 82)
(393345, 46)


In [7]:
# model = svm_builder.train_svm_model(X_pca1, Y)
model_X = X_pca1
model_y = y
print(model_X.shape, model_y.shape)

(393345, 82) (393345,)


In [8]:
from logistic_builder import *
train_logistic_model_with_cross_validation(model_X, model_y)

D:\Anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
D:\Anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
D:\Anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
D:\A

Cross-Validation scores: [0.90092793 0.88097115 0.87459006 0.89058091 0.84156604 0.89866273
 0.88483246 0.84829918 0.88147659 0.87842579]
Mean score: 0.8780332825645123


In [1]:
import logistic_builder
importlib.reload(logistic_builder)
model_logistic = logistic_builder.train_logistic_model(model_X, model_y)
model_logistic = logistic_builder.train_logistic_model(X_scaled, y)

NameError: name 'importlib' is not defined

In [ ]:
from bayes_builder import train_bayes_model
model_bayes = train_bayes_model(model_X, model_y)
model_bayes = train_bayes_model(X_scaled, y)

In [ ]:
model_svm = svm_builder.train_svm_model(model_X, model_y)
model_svm = svm_builder.train_svm_model(X_scaled, y)